In [1]:
import akshare as ak
import pandas as pd
stock_zh_a_spot_df = ak.stock_zh_a_spot_em()

/Users/yangjingjing/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
filter_data = stock_zh_a_spot_df[['代码', '名称', '最新价', '涨跌幅','市盈率-动态', '市净率', '总市值']]

selected_stocks = pd.DataFrame(filter_data)

# 保存到Excel文件
selected_stocks.to_excel('filtered_stocks.xlsx', index=False)

print("Stocks with market cap less than 200 billion exported to Excel successfully.")

Stocks with market cap less than 200 billion exported to Excel successfully.


In [3]:
import re
# & ~stock_zh_a_spot_df['名称'].str.contains('ST') & ~stock_zh_a_spot_df['名称'].str.contains('\*ST')]
filtered_stocks = {'name': [], 'code': [], 'market_cap': []}
for index,stock_item in enumerate(stock_zh_a_spot_df['总市值']):
    stock_name = stock_zh_a_spot_df['名称'][index]
    market_cap_item = stock_item/100000000
    stock_code = stock_zh_a_spot_df['代码'][index]
    if not re.search(r'st|\*st', stock_name, re.IGNORECASE) and  market_cap_item < 100 and re.search(r'600|601|603|000|001|002', stock_code[0:3]):
            filtered_stocks['name'].append(stock_zh_a_spot_df['名称'][index])
            filtered_stocks['code'].append(stock_code)
            filtered_stocks['market_cap'].append(market_cap_item)

print(len(filtered_stocks['name']))
print(len(filtered_stocks['code']))


1913
1913


In [5]:

limit_up_stocks = {'name': [], 'code': [], 'market_cap': [], 'type': [], 'rise_percent': []}
for index,stock_code in enumerate(filtered_stocks['code'][0:1000]):
    print(stock_code)
    stock_hist = ak.stock_zh_a_hist(symbol=stock_code, start_date="20240111", end_date='20240112')
    if stock_hist.empty:
        pass
    elif stock_hist['收盘'][1] == stock_hist['最高'][1]:
        stock_rise_percent = (stock_hist['收盘'][1] - stock_hist['收盘'][0]) / stock_hist['收盘'][0] * 100
        print(index, stock_code,filtered_stocks['name'][index])
        if stock_rise_percent > 9.9:
            type = ''
            if stock_hist['收盘'][1] == stock_hist['开盘'][1] and stock_hist['收盘'][1] > stock_hist['最低'][1]:
                type = 'T1'
                print('T1',index,stock_code, stock_rise_percent)
            elif stock_hist['收盘'][1] >= stock_hist['开盘'][1] and stock_hist['开盘'][1] > stock_hist['最低'][1]:
                type = 'T2'
                print('T2',index,stock_code, stock_rise_percent)
            else:
                type = 'T3'
                print('T3',index,stock_code, stock_rise_percent)
                
            limit_up_stocks['name'].append(filtered_stocks['name'][index])
            limit_up_stocks['code'].append(filtered_stocks['code'][index])
            limit_up_stocks['market_cap'].append(filtered_stocks['market_cap'][index])
            limit_up_stocks['type'].append(type)
            
selected_stocks = pd.DataFrame(limit_up_stocks)

# 保存到Excel文件
selected_stocks.to_excel('filtered_stocks.xlsx', index=False)

print("Stocks with market cap less than 200 billion exported to Excel successfully.")


603312
600819
1 600819 耀皮玻璃
T2 1 600819 10.097087378640769
600744
2 600744 华银电力
T2 2 600744 10.086455331412093
000017
3 000017 深中华A
T3 3 000017 10.08403361344537
600565
4 600565 迪马股份
T2 4 600565 10.067114093959725
002651
5 002651 利君股份
T2 5 002651 10.044977511244376
002606
6 002606 大连电瓷
T2 6 002606 10.025380710659899
001212
7 001212 中旗新材
T2 7 001212 10.021052631578943
000638
8 000638 万方发展
T2 8 000638 10.01430615164521
603628
9 603628 清源股份
T2 9 603628 10.013060513713542
002140
10 002140 东华科技
T2 10 002140 10.011778563015309
603276
11 603276 恒兴新材
T3 11 603276 10.007757951900691
002868
12 002868 绿康生化
T3 12 002868 10.004426737494473
603109
13 603109 神驰机电
T2 13 603109 9.999999999999993
002641
14 002641 公元股份
T3 14 002641 10.000000000000005
603097
15 603097 江苏华辰
T2 15 603097 9.991638795986608
002708
16 002708 光洋股份
T1 16 002708 9.98937300743889
603038
17 603038 华立股份
T3 17 603038 9.98509687034277
603955
18 603955 大千生态
T3 18 603955 9.981740718198422
603131
19 603131 上海沪工
T2 19 603131 9.97846374730

ValueError: All arrays must be of the same length